# Setup

In [5]:
# Shared "communication" directory on Drive.
shared_directory = '/content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/'

my_prefix = "find_"
other_prefix = "camo_"

my_suffix =  ".txt"
other_suffix = ".jpeg"

import time
from os import listdir
from os import remove
from os.path import join
from os.path import split
from os.path import isfile

# After c++ tests

In [6]:

def run_test():
    print('Start run in', shared_directory )
    list_unexpected_files(shared_directory)
    step = 0;
    while True:
        performStep(step, shared_directory)
        step += 1

def performStep(step, directory):
    waitForReply(step, shared_directory)
    print('Write file', step)
    writeTestFile(step, shared_directory)
    deleteMyFile(step - 1, shared_directory)

# Mock version of writing file for given step.
def writeTestFile(step, directory):
    pathname = makeMyPathname(step, directory)
    file = open(pathname,"w")
    # file.write(str(step))
    file.write(str(step) + " " + str(step))
    print("wrote test file  ", makeMyPathname(step, directory))

# Delete the given file, usually after having written the next one.
def deleteMyFile(step, directory):
    # fs::remove(makeMyPathname(step, directory));
    path = makeMyPathname(step, directory)
    if isfile(path):
        remove(path)

# Form pathname for file of given step number from the "other" agent.
def makeOtherPathname(step, directory):
    # return directory + other_prefix + str(step) + ".txt"
    return directory + other_prefix + str(step) + other_suffix

# Form pathname for file of given step number from "this" agent.
def makeMyPathname(step, directory):
    # return directory + my_prefix + str(step) + ".txt"
    return directory + my_prefix + str(step) + my_suffix

# Wait until other agent's file for given step appears.
def waitForReply(step, directory):
    print('start waiting for', makeOtherPathname(step, directory))
    start_time = time.time()
    while not isFilePresent(makeOtherPathname(step, directory)):
        time.sleep(2)  # wait 2 sec
    print('done waiting for ', makeOtherPathname(step, directory))
    print('Elapsed time:', int(time.time() - start_time), 'seconds.')

# Like fs::exists() but for unknown reasons, that does not
# seem to work for newly created files on G Drive.
#
# TODO Why? This version works on G Drive, but it seems simply
#      calling fs::exists() should be enough.
#
def isFilePresent(file):
    result = False
    (directory, filename) = split(file)
    for i in listdir(directory):
        if i == filename:
            result = True
    return result

# isFilePresent(comm_directory + 'foo.txt')
# isFilePresent(comm_directory + 'bar.txt')
# waitForReply(1, comm_directory)

# Actually I guess the counterparty may have already written its first...
def list_unexpected_files(directory):
    directory_contents = listdir(directory)
    if directory_contents:
        print('Unexpected files:', directory_contents)

# Run test

In [7]:
run_test()

Start run in /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/
start waiting for /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/camo_0.jpeg
done waiting for  /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/camo_0.jpeg
Elapsed time: 38 seconds.
Write file 0
wrote test file   /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/find_0.txt
start waiting for /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/camo_1.jpeg
done waiting for  /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/camo_1.jpeg
Elapsed time: 50 seconds.
Write file 1
wrote test file   /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/find_1.txt
start waiting for /content/drive/My Drive/PredatorEye/evo_camo_vs_static_fcd/camo_2.jpeg


KeyboardInterrupt: ignored